In [1]:
# Standard import
import os

# 3rd-party import
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

# Local import

c:\Users\chun\Desktop\Projects\NaturalLanguageProcessing\Chatbot_RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
CHROMA_PATH = "chroma"
PROMPT_TEMPLATE = """
Answer the question based only on the following context:{context}
Answer the question based on the above context:{question}
"""

In [3]:
def query_database(query_text: str) -> None:
    """
    Query the Chroma database and get a response from the Gemini model.
    
    Args:
        query_text: The question to ask the RAG system
    """
    # Prepare db
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    db = Chroma(
        persist_directory=CHROMA_PATH, 
        embedding_function=embeddings
    )

    # Search db
    results = db.similarity_search_with_relevance_scores(query_text, k=3)

    # See the top scorers
    # print(f"Found {len(results)} results:")
    # for i, (doc, score) in enumerate(results):
    #     print(f"Result {i+1} - Score: {score:.4f}")
    #     print(f"Content: {doc.page_content[:200]}...")
    #     print(f"Source: {doc.metadata.get('source', 'Unknown')}")
    #     print("-" * 50)

    # Apply threshold
    threshold = 0.1
    if len(results) == 0 or results[0][1] < threshold:
        print(f"Unable to find matching results with threshold {threshold}. Highest score: {results[0][1] if results else 'N/A'}")
        return
    
    # Prepare context and prompt
    context = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context, question=query_text)

    # Get response from model
    model = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-exp",
        google_api_key=os.getenv('GEMINI_API_KEY')
    )
    response = model.invoke(prompt)
    response_text = response.content

    # Format and display results
    sources = [doc.metadata.get("source", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)

In [4]:
def interactive_query() -> None:
    """
    Run an interactive loop to query the database multiple times.
    """
    print("RAG System Ready!")
    print("Type 'quit' or 'exit' to stop.\n")
    
    while True:
        # Get user input
        query_text = input("Enter your question: ").strip()
        
        # Check for exit conditions
        if query_text.lower() in ['q']:
            print("Goodbye!")
            break
            
        if not query_text:
            print("Please enter a question.\n")
            continue
            
        print(f"\nQuery: {query_text}")
        print("-" * 50)
        
        # Process the query
        query_database(query_text)
        print("\n" + "=" * 50 + "\n")

In [5]:
def main() -> None:
    interactive_query()

In [6]:
main()

RAG System Ready!
Type 'quit' or 'exit' to stop.


Query: can i bring my cats to the hotel with me
--------------------------------------------------
Response: No, pets are not allowed in the residence.
Sources: ['..\\data\\FAQ.txt', '..\\data\\FAQ.txt', '..\\data\\Refund_Policy.txt']



Query: may i know the provided facilities or infrastructures
--------------------------------------------------
Response: The provided facilities in Viia Residence are: Inﬁnity Sky Pool, Gourmet lounge, Co-Working Space, and self-service laundry.
Sources: ['..\\data\\FAQ.txt', '..\\data\\FAQ.txt', '..\\data\\FAQ.txt']



Query: why do cats like to stay inside a box
--------------------------------------------------
Unable to find matching results with threshold 0.1. Highest score: -0.023076141088716584




C:\Users\chun\AppData\Local\Temp\ipykernel_10088\4187936716.py:16: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='2c89977e-5560-4bf5-9e09-480c61d5f445', metadata={'start_index': 1237, 'source': '..\\data\\Refund_Policy.txt'}, page_content='3. What Travel Issues are covered\n\nThe term "Travel Issue" refers to these situations:\n\n(a)\u2003Host cancels the reservation prior to check-in.\n\n(b)\u2003Host fails to provide access to the accommodations.\n\n(c)\u2003Host fails to disclose in the Listing that the Host, another person, or a pet will be present during the stay.\n\n(d)\u2003Accommodations are not habitable at check-in for any of the following reasons:\n\nThey are not reasonably clean and sanitary, including bedding and towels. They contain safety or health hazards. They contain pests. (e)\u2003Listing contains a material inaccuracy such as:'), -0.023076141088716584), (Document(id='bbb5ca3b-9e90-42f1-85b9-93ecbeed3d09', metadata={'start_index': 1325, 's

Goodbye!
